## K-Means Clustering 후 같은 클러스터 내의 기사를 cosine similarity를 통한 유사도 도출

In [18]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
from datetime import date, timedelta
import io
import time
import sqlite3
import pandas as pd
import csv
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
df = pd.read_csv('news_content_processed_data.csv', index_col=0)
df = df.dropna().reset_index()

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# TfidfVectorizer를 사용하여 문서를 벡터화한다.
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['content'])

# k-means 클러스터링을 수행한다.
k = 20  # 클러스터의 개수를 지정한다.
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# 클러스터링 결과를 출력한다.
for i in range(k):
    cluster_data = df[df['cluster'] == i]['content']
    print(f'Cluster {i + 1}:\n')
    for j, content in enumerate(cluster_data.head()):
        print(f"{j + 1}. {content}\n")
    print('=' * 50 + '\n')


c:\Users\kms10\anaconda3\envs\py3_11\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster 1:

1. 원희룡 국토교통부 장관은 1일 “주택가격은 수요와 공급에 따라 오르고 내리는 것이라 가격 자체는 국가가 어쩔 수 있는 부분은 아니지만 거래가 단절되다시피 하는 것은 문제기 때문에 최소한의 거래 활성화를 위해 국가가 지원해야 한다”고 말했다.
원 장관은 1일 KTV(국민방송) 국정 대담 ‘국민이 묻고 장관이 답하다’에 출연해 방청객이 “집값이 떨어지는 것도 문제지만, 거래가 안 돼 이사하기도 어려운 상황인데 규제를 풀어줄 생각이 있느냐”고 묻자 이처럼 답했다.
원 장관은 “주택 가격 자체는 국가가 어쩔 수 있는 부분이 아니다”며 “가격은 수요와 공급에 따라 오르고 내리는 것이기 때문에 집값 자체는 죄가 아니다”고 말했다.
이어 “그런데 거래 자체가 단절되다 보니 이사 가야 하는 분들, 청약에 당첨됐는데 대출이 끊어져서 기회를 놓칠 위기에 있는 분들, 실거주 내지는 이사나 자산형성을 하는 데 있어서 자연스럽게 진행되던 게 갑자기 끊어지는 부분은 국가가 지원해야 한다”고 강조했다.
원 장관은 “최소한의 거래 활성화를 위해 규제를 풀려고 하는 것”이라며 “금융, 조정지역 등 이런 부분을 어느 정도 풀어야 할지 검토하고 있고 거래와 미분양 최악의 상황을 해소하기 위해 정부가 준비를 다 해놓고 있는 만큼 조금만 기다려달라”고 말했다.
심야 택시 문제와 관련 “단거리 손님을 기피하는 문제를 없애기 위해서 ‘목적지 미표시’를 법적으로 강제하려고 한다”고 밝혔다.
안전운임제 관련해서는 “번호판 장사를 하는 중간 빨대, 지입제를 개선하고 진짜 약자인 화물연대 비조합원들을 보호하고 운행기록계를 단속해 진짜 안전을 보장하는 내용을 답아 협의체를 만들어 논의하고 국회에서 처리할 것”이라고 약속했다.

2. 홍준표 10.3%, 오세훈 6.2% 順
차기 지도자 적합도 여론조사에서 더불어민주당 이재명 대표가 1위에 올랐다. 한동훈 법무부 장관은 2위를 기록했다. 여권에선 홍준표 대구시장과 원희룡 국토교통부 장관의 약진이 돋보였다.
 
 
 
  이재명 더불어민주

In [21]:
df

,index,date,press,title,url,content,cluster
0,0,20230101,32,“아내와 함께 떠난 딸…마지막이 될 줄 몰랐다”,https://n.news.naver.com/article/032/000319644...,과천 방음터널 화재로 모녀 숨져\n딸은 뇌졸중 엄마 지극히 간병\n하루아침에 아내와...,12
1,1,20230101,32,"뜸 들이다 끝난 ‘더 글로리’···복수한다며, 남자가 왜 거기서 나와[리뷰]",https://n.news.naver.com/article/032/000319637...,김은숙 작가와 배우 송혜교의 첫 장르물로 기대를 모았던 넷플릭스 드라마 <더 글로리...,11
2,2,20230101,32,이것은 발 토시가 아니다…세련미 더해 돌아온 ‘라떼는’ 패션,https://n.news.naver.com/article/032/000319630...,"그해의 유행을 속성으로 배울 수 있는 시간, 바로 연말 시상식 무대다. 지난 16일...",11
3,3,20230101,32,"경찰, ‘방음터널 화재’ 폐기물 운반용 집게 트럭 소유 업체 압수수색",https://n.news.naver.com/article/032/000319636...,5명의 사망자가 발생한 ‘방음터널 화재 사고’를 수사 중인 경찰이 폐기물 운반용 집...,12
4,4,20230101,32,다시 만난 ‘자낳세’…“월급만으로 살 수 없단 생각엔 변함 없어”,https://n.news.naver.com/article/032/000319644...,“주식 투자를 해야 한다는 생각에는 변함이 없어요. 투자로 계속 자산을 불려 나가야...,19
...,...,...,...,...,...,...,...
48497,48502,20231014,20,캄캄한 새벽 도로에 누워있던 30대 택시에 깔려 숨져,https://n.news.naver.com/article/020/000352538...,제주에서 새벽시간 도로에 누워있던 30대 남성이 택시에 깔려 숨지는 사고가 발생했다...,12
48498,48503,20231014,20,‘코끼리는 생각하지 마!’…저출산 프레임에서 빠져나오자[이미지의 포에버 육아],https://n.news.naver.com/article/020/000352537...,‘포(four)에버 육아’는 네 명의 자녀를 키우며 직장생활을 병행하고 있는 가 일...,19
48499,48504,20231014,20,野지도부 “가결파 전부 아닌 해당 행위자만 징계 논의”,https://n.news.naver.com/article/020/000352533...,이르면 내주 이재명 복귀 앞두고\n징계 대상 줄여 잡음 최소화 의도\n더불어민주당 ...,2
48500,48505,20231014,20,이스라엘 “며칠내 가자서 작전” 진입 초읽기,https://n.news.naver.com/article/020/000352531...,가자 주민 110만명에 대피 통첩\n현지언론 “이르면 14일 진입 가능성”\n이란-...,3


In [22]:
# 각 클러스터 내에서의 코사인 유사도를 계산합니다.
cluster_cosine_similarities = []
unique_clusters = df['cluster'].unique()

for cluster_number in tqdm(unique_clusters):
    cluster_indices = df[df['cluster'] == cluster_number].index
    cluster_cosine_similarities.append(cosine_similarity(X[cluster_indices], X[cluster_indices]))


  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:22<00:00,  1.15s/it]


In [27]:
cluster_cosine_similarities

[array([[1.        , 0.45870975, 0.29425577, ..., 0.00825878, 0.02310436,
         0.01653622],
        [0.45870975, 1.        , 0.40409331, ..., 0.00979818, 0.04528311,
         0.03194097],
        [0.29425577, 0.40409331, 1.        , ..., 0.00969285, 0.03660315,
         0.02958734],
        ...,
        [0.00825878, 0.00979818, 0.00969285, ..., 1.        , 0.0280306 ,
         0.04629844],
        [0.02310436, 0.04528311, 0.03660315, ..., 0.0280306 , 1.        ,
         0.0418951 ],
        [0.01653622, 0.03194097, 0.02958734, ..., 0.04629844, 0.0418951 ,
         1.        ]]),
 array([[1.00000000e+00, 8.06486087e-03, 2.09035126e-03, ...,
         1.60524346e-03, 1.02873045e-03, 2.71357284e-04],
        [8.06486087e-03, 1.00000000e+00, 6.31054526e-03, ...,
         4.06801341e-03, 4.85671317e-03, 2.72430260e-03],
        [2.09035126e-03, 6.31054526e-03, 1.00000000e+00, ...,
         5.01885881e-03, 4.54584393e-03, 1.49249284e-03],
        ...,
        [1.60524346e-03, 4.06801341e